# Optimal tau as a function of the image and noise level

In [ ]:
local_debug = True

In [ ]:
import time
from collections import namedtuple

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

import scipy.ndimage

try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker

#from ipywidgets import *
try:
    import cv2
except:
    !pip install cv2
    import cv2
    
#import kernels
try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io

try:
    import information_theory as IT
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    import information_theory as IT

import utils

In [ ]:
# apt install cm-super-minimal
# apt install dvipng
plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})
#rc('text', usetex=True)
#rc('font', family='serif')
#plt.rc('text.latex', preamble=r'\usepackage{amsmath} `...')
#mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath} \usepackage{amsfonts}'

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

In [ ]:
try:
    import denoising.image.gaussian as denoising
except:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
    import denoising.image.gaussian as denoising

### barb

In [ ]:
Args = namedtuple("args", ["input1", "input2"])
args = Args("http://www.hpca.ual.es/~vruiz/images/barb.png", "http://www.hpca.ual.es/~vruiz/images/lake.png")
#args = Args("http://www.hpca.ual.es/~vruiz/images/zelda.png")
#args = Args("http://www.hpca.ual.es/~vruiz/images/boats.png")
#args = Args("http://www.hpca.ual.es/~vruiz/images/lake.png")

In [ ]:
image = skimage_io.imread(args.input1)  # Ground Truth
X1 = image
X1.shape

In [ ]:
plt.title(r"$\mathbf{X}_1=\mathrm{Barb}$")
utils.imshow(X1)
plt.savefig('barb.pdf', bbox_inches='tight')

### lake

In [ ]:
image = skimage_io.imread(args.input2)  # Ground Truth
X2 = image
X2.shape

In [ ]:
plt.title(r"$\mathbf{X}_2=\mathrm{Lake}$")
utils.imshow(X2)
plt.savefig('lake.pdf', bbox_inches='tight')

## Zero-mean additive Gaussian noise (0MAGN)
\begin{equation}
  Y = X + N,~N\sim{\mathcal N}(\mu=0, \sigma).
\end{equation}

#### 0MAGN_barb

In [ ]:
STD_DEV = 40 # Gaussian noise level
#string = r"$\mathrm{Barb} + \mathbf{N}, \mathbf{N}\sim \mathcal{N}" + rf"(\mu=0, \sigma={std_dev})$"
string  = r"$\hat{\mathbf{X}}_1="
string += r"[\mathbf{X}_1 + \mathbf{N}_{\mathcal{N}(\sigma="
string += str(STD_DEV)
string += r")}"
Y1 = utils.add_0MWGN(X1, STD_DEV)
PSNR1 = IT.distortion.PSNR(Y1, X1)
string += rf"], {PSNR1:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
#Y = X + np.random.normal(loc=0, scale=STD_DEV, size=X.shape).reshape(X.shape)
#Y = np.clip(a = Y, a_min=0, a_max=255)
#plt.imshow(np.clip(a = Y, a_min=0, a_max=255), cmap="gray")
utils.imshow(Y1)
plt.savefig('0MAGN_barb.pdf', bbox_inches='tight')

#### 0MAGN_lake

In [ ]:
STD_DEV = 40 # Gaussian noise level
#string = r"$\mathrm{Barb} + \mathbf{N}, \mathbf{N}\sim \mathcal{N}" + rf"(\mu=0, \sigma={std_dev})$"
string  = r"$\hat{\mathbf{X}}_2="
string += r"[\mathbf{X}_2 + \mathbf{N}_{\mathcal{N}(\sigma="
string += str(STD_DEV)
string += r")}"
Y2 = utils.add_0MWGN(X2, STD_DEV)
PSNR2 = IT.distortion.PSNR(Y2, X2)
string += rf"], {PSNR2:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
#Y2 = utils.add_0MWGN(X2, STD_DEV)
#Y = X + np.random.normal(loc=0, scale=STD_DEV, size=X.shape).reshape(X.shape)
#Y = np.clip(a = Y, a_min=0, a_max=255)
#plt.imshow(np.clip(a = Y, a_min=0, a_max=255), cmap="gray")
utils.imshow(Y2)
plt.savefig('0MAGN_lake.pdf', bbox_inches='tight')

#### GD_0MAGN_barb

In [ ]:
denoiser = denoising.Monochrome_Denoising(logger)

def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

In [ ]:
TAU = 1.5 # Signa kernel

In [ ]:
sigma = np.array([TAU, TAU])
kernel = [None]*2
kernel[0] = utils.get_gaussian_kernel(sigma[0])
kernel[1] = utils.get_gaussian_kernel(sigma[1])
print(np.sum(kernel[0]))
plt.plot(kernel[0])
plt.show()

In [ ]:
denoised1 = denoiser.filter(Y1, kernel)

In [ ]:
PSNR1 = IT.distortion.PSNR(denoised1, X1)

In [ ]:
#plt.title(r"$\mathrm{GD}_{\text{" + str(sigma_kernel) + r"}}(\mathrm{Barb} + \mathbf{N}_{\mathcal{N}}), \mathbf{N}\sim \mathcal{N}_{}" + rf"(\mu=0, \sigma={std_dev})," + r"\mathrm{PSNR}=" + f"{PSNR:.2f}" + r"~\mathrm{dB}$")
string  = r"$"
string += r"\mathrm{GD}"
string += r"_\text{"
string += str(TAU)
string += r"}"
string += r"(\hat{\mathbf{X}}_1)"
string += rf", {PSNR1:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
#plt.imshow(denoised, cmap="gray")
utils.imshow(denoised1)
plt.savefig('GD_0MAGN_barb.pdf', bbox_inches='tight')

#### GD_0MAGN_lake

In [ ]:
denoised2 = denoiser.filter(Y2, kernel)

In [ ]:
PSNR2 = IT.distortion.PSNR(denoised2, X2)

In [ ]:
string  = r"$"
string += r"\mathrm{GD}"
string += r"_\text{"
string += str(TAU)
string += r"}"
string += r"(\hat{\mathbf{X}}_2)"
string += rf", {PSNR1:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
#plt.imshow(denoised, cmap="gray")
utils.imshow(denoised2)
plt.savefig('GD_0MAGN_lake.pdf', bbox_inches='tight')

#### GD_PCC_0MAGN_barb and GD_PCC_0MAGN_lake
Shows the impact of the length of the filter (sigma_kernel) for different noise levels (std_dev).

In [ ]:
sigmas_kernel = [0.25, 0.35, 0.5, 0.65, 0.75, 0.9, 1.0, 1.5]
sigmas_kernel = [0.5 + i/5 for i in range(30)]
std_devs = [i for i in range(0, 120, 20)]
PCC_curves1 = []
PCC_curves2 = []
#sigma_index = 1
#for std_dev in range(10, 50, int((50-10)/len(sigmas_kernel))):
for std_dev in std_devs:
    Y1 = utils.add_0MWGN(X1, std_dev)
    Y2 = utils.add_0MWGN(X2, std_dev)
    #Y = X + np.random.normal(loc=0, scale=std_dev, size=X.shape).reshape(X.shape)
    #Y = np.clip(a = Y, a_min=0, a_max=255)
    PCC_curve1 = []
    PCC_curve2 = []
    for sigma_kernel in sigmas_kernel: #range(5, 20, 1):
        #sigma_kernel /= 10
        sigma = np.array([sigma_kernel, sigma_kernel])
        kernel = [None]*2
        kernel[0] = utils.get_gaussian_kernel(sigma[0])
        kernel[1] = utils.get_gaussian_kernel(sigma[1])
        denoised1 = denoiser.filter(Y1, kernel)
        denoised2 = denoiser.filter(Y2, kernel)
        #PSNR = IT.distortion.PSNR(denoised, X)
        PCC1 = np.corrcoef(denoised1.flatten(), X1.flatten())[0, 1]
        PCC2 = np.corrcoef(denoised2.flatten(), X2.flatten())[0, 1]
        print(std_dev, sigma_kernel, PCC1, PCC2)
        PCC_curve1.append(PCC1)
        PCC_curve2.append(PCC2)
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    PCC_curves1.append(PCC_curve1)
    PCC_curves2.append(PCC_curve2)
    #sigma_index += 1
    print()

In [ ]:
#plt.title(r'$\mathbf{X}=\mathrm{Barb}$')
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
for i in range(len(PCC_curves1)):
    #plt.plot([j/10 for j in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}$")
    plt.plot(sigmas_kernel,
             PCC_curves1[i],
             label=rf"$\sigma={std_devs[i]}$"
             + r"$, \tau^*="
             + rf"{sigmas_kernel[np.argmax(PCC_curves1[i])]}$")
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}_1, \mathrm{GD}_\tau(\mathbf{X}_1+\mathbf{N}_{\mathcal{N}(\sigma)}))$")
#plt.xlabel(r"$\tau~(\text{Standard deviation of the Gaussian kernel})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GD_PCC_0MMPG_barb.pdf", bbox_inches='tight')

In [ ]:
#plt.title(r'$\mathbf{X}=\mathrm{Lake}$')
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
for i in range(len(PCC_curves2)):
    #plt.plot([j/10 for j in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}$")
    plt.plot(sigmas_kernel,
             PCC_curves2[i],
             label=rf"$\sigma={std_devs[i]}$"
             + r"$, \tau^*="
             + rf"{sigmas_kernel[np.argmax(PCC_curves2[i])]}$")
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}_2, \mathrm{GD}_\tau(\mathbf{X}_2+\mathbf{N}_{\mathcal{N}(\sigma)}))$")
#plt.xlabel(r"$\tau~(\text{Standard deviation of the Gaussian kernel})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GD_PCC_0MMPG_lake.pdf", bbox_inches='tight')

## Poisson noise (shot noise)
\begin{equation}
  Y=\frac{N}{\gamma},~N∼\mathcal{P}(λ=\gamma X),
\end{equation}
where:
* $X\ge 0$.
* $\lambda\in{\mathbb R}\ge 0$ is the rate of the Poisson distribution, resulting that $\lambda={\mathbb E}(N)=\text{Var}(N)$.

In [ ]:
GAMMA = 0.15

#### Poisson_barb

In [ ]:
string  = r"$\hat{\mathbf{X}}_1=[\mathbf{N}"
string += r"_{\mathcal{P}(\lambda="
string += rf"{GAMMA}"
string += r"\cdot\mathbf{X}_1)}"
string += rf"/{GAMMA}]"
Y1 = utils.generate_PN(X1, GAMMA)
#Y = np.random.poisson(X * gamma) / gamma
#Y = X + np.random.poisson(0.5*X)
#from skimage.util import random_noise
#Y = random_noise(X, mode="poisson")
Y1 = np.clip(a = Y1, a_min=0, a_max=255)
PSNR1 = IT.distortion.PSNR(Y1, X1)
string += rf", {PSNR1:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
#plt.imshow(Y, cmap="gray")
utils.imshow(Y1)
plt.savefig('Poisson_barb.pdf', bbox_inches='tight')

#### Poisson_lake

In [ ]:
string  = r"$\hat{\mathbf{X}}_2=[\mathbf{N}"
string += r"_{\mathcal{P}(\lambda="
string += rf"{GAMMA}"
string += r"\cdot\mathbf{X}_2)}"
string += rf"/{GAMMA}]"
Y2 = utils.generate_PN(X2, GAMMA)
#Y = np.random.poisson(X * gamma) / gamma
#Y = X + np.random.poisson(0.5*X)
#from skimage.util import random_noise
#Y = random_noise(X, mode="poisson")
Y2 = np.clip(a = Y2, a_min=0, a_max=255)
PSNR2 = IT.distortion.PSNR(Y2, X2)
string += rf", {PSNR2:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
#plt.imshow(Y, cmap="gray")
utils.imshow(Y2)
plt.savefig('Poisson_lake.pdf', bbox_inches='tight')

#### GD_Poisson_barb

In [ ]:
denoised1 = denoiser.filter(Y1, kernel)

In [ ]:
PSNR1 = IT.distortion.PSNR(denoised1, X1)

In [ ]:
#plt.title(r"$\mathrm{GD}_{\text{" + str(sigma_kernel) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\cdot\mathrm{Barb})$")
string  = r"$"
string += r"\mathrm{GD}"
string += r"_\text{"
string += str(TAU)
string += r"}"
string += r"(\hat{\mathbf{X}}_1)"
string += rf", {PSNR1:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
utils.imshow(denoised1)
plt.savefig('GD_Poisson_barb.pdf', bbox_inches='tight')

#### GD_Poisson_lake

In [ ]:
denoised2 = denoiser.filter(Y2, kernel)

In [ ]:
PSNR2 = IT.distortion.PSNR(denoised2, X2)

In [ ]:
string  = r"$"
string += r"\mathrm{GD}"
string += r"_\text{"
string += str(TAU)
string += r"}"
string += r"(\hat{\mathbf{X}}_2)"
string += rf", {PSNR2:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
utils.imshow(denoised2)
plt.savefig('GD_Poisson_lake.pdf', bbox_inches='tight')

#### GD_PCC_Poisson_barb and GD_PCC_Poisson_lake

In [ ]:
for gamma in range(len(sigmas_kernel)):
    print((1+gamma)/40)

In [ ]:
print([(0.25 + i/10) for i in range(20)])

In [ ]:
#sigmas_kernel = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
sigmas_kernel = [(0.25 + i/10) for i in range(20)]
PCC_curves1 = []
PCC_curves2 = []
#sigma_index = 1
#for _lambda in range(10, 0, -int((10)/len(sigmas_kernel))):
#for _lambda in range(len(sigmas_kernel)):
for gamma in range(8):
    #gamma /= 40
    gamma = (1+gamma)/10
    PCC_curve1 = []
    PCC_curve2 = []
    for sigma_kernel in sigmas_kernel: #range(5, 20, 1):
        #sigma_kernel /= 10
        sigma = np.array([sigma_kernel, sigma_kernel])
        kernel = [None]*2
        kernel[0] = utils.get_gaussian_kernel(sigma[0])
        kernel[1] = utils.get_gaussian_kernel(sigma[1])
        Y1 = np.random.poisson(X1.astype(np.float32) * gamma) / gamma
        Y1 = np.clip(a = Y1, a_min=0, a_max=255)
        Y2 = np.random.poisson(X2.astype(np.float32) * gamma) / gamma
        Y2 = np.clip(a = Y2, a_min=0, a_max=255)
        denoised1 = denoiser.filter(Y1, kernel)
        denoised2 = denoiser.filter(Y2, kernel)
        #PSNR = IT.distortion.PSNR(denoised, X)
        PCC1 = np.corrcoef(denoised1.flatten(), X1.flatten())[0, 1]
        PCC2 = np.corrcoef(denoised2.flatten(), X2.flatten())[0, 1]
        print(f"gamma={gamma}", f"sigma_kernel={sigma_kernel}", f"PCC1={PCC1}", f"PCC2={PCC2}")
        PCC_curve1.append(PCC1)
        PCC_curve2.append(PCC2)
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    PCC_curves1.append(PCC_curve1)
    PCC_curves2.append(PCC_curve2)
    #sigma_index += 1
    print()

In [ ]:
len(sigmas_kernel)

In [ ]:
#plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
for i in range(len(PCC_curves1)):
    plt.plot(sigmas_kernel,
             PCC_curves1[i],
             label=rf"$\gamma={(1+i)/40}$"
             + r"$, \tau^*="
             + rf"{sigmas_kernel[np.argmax(PCC_curves1[i])]}$")
    #plt.plot([i/10 for i in range(5, 20, 1)], curves[i], label=rf"$\lambda={(10-i)/40}\cdot" + r"\mathrm{Barb}$")
string  = r"$"
string += r"\mathrm{PCC}"
string += r"(\mathbf{X}_1, "
string += r"\mathrm{GD}_\tau(\mathbf{N}_"
string += r"{\mathcal{P}(\gamma\mathbf{X}_1)}/\gamma))$"
plt.ylabel(string)
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GD_PCC_Poisson_barb.pdf", bbox_inches='tight')

In [ ]:
#plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
for i in range(len(PCC_curves1)):
    plt.plot(sigmas_kernel,
             PCC_curves2[i],
             label=rf"$\gamma={(1+i)/40}$"
             + r"$, \tau^*="
             + rf"{sigmas_kernel[np.argmax(PCC_curves2[i])]}$")
    #plt.plot([i/10 for i in range(5, 20, 1)], curves[i], label=rf"$\lambda={(10-i)/40}\cdot" + r"\mathrm{Barb}$")
string  = r"$"
string += r"\mathrm{PCC}"
string += r"(\mathbf{X}_2, "
string += r"\mathrm{GD}_\tau(\mathbf{N}_"
string += r"{\mathcal{P}(\gamma\mathbf{X}_2)}/\gamma))$"
plt.ylabel(string)
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GD_PCC_Poisson_lake.pdf", bbox_inches='tight')

## Zero-mean Mixed Poisson-Gaussian (0MMPG) noise
\begin{equation}
  Y = N_{\mathcal{P}(\gamma X)}/\gamma + N_{\mathcal{N(\sigma)}}
\end{equation}
where
\begin{equation}
N_{\mathcal{P}(\gamma X)}\sim\mathcal{P}(λ=\gamma X)
\end{equation}
and
\begin{equation}
N_{\mathcal{N(\sigma)}}\sim{\mathcal N}(\mu=0, \sigma^2).
\end{equation}

#### 0MMPG_barb

In [ ]:
string  = r'$\hat{\mathbf{X}}_1=['
string += r"\mathbf{X}_1 + \mathbf{N}_{\mathcal{N}"
string += r"(\sigma"
string += f"={STD_DEV})"
string += r'}'
string += r" + \mathbf{N}_{\mathcal{P}"
string += r"(\lambda"
string += f"={GAMMA}"
string += r'\cdot\mathbf{X}_1)}'
string += f"/{GAMMA}"
Y1 = utils.generate_MPGN(X1, STD_DEV, GAMMA)
Y1 = np.clip(a = Y1, a_min=0, a_max=255)
PSNR1 = IT.distortion.PSNR(Y1, X1)
string += rf"], {PSNR1:.2f}"
string += r"~\mathrm{dB}"
string += '$'
plt.title(string) 
#Y = MPG_noise(X, STD_DEV, GAMMA).reshape(X.shape)
utils.imshow(Y1)
plt.savefig('0MMPG_barb.pdf', bbox_inches='tight')

#### 0MMPG_lake

In [ ]:
GAMMA = 0.15
string  = r'$\hat{\mathbf{X}}_2=['
string += r"\mathbf{X}_2 + \mathbf{N}_{\mathcal{N}"
string += r"(\sigma"
string += f"={STD_DEV})"
string += r'}'
string += r" + \mathbf{N}_{\mathcal{P}"
string += r"(\lambda"
string += f"={GAMMA}"
string += r'\cdot\mathbf{X}_2)}'
string += f"/{GAMMA}"
Y2 = utils.generate_MPGN(X2, STD_DEV, GAMMA)
Y2 = np.clip(a = Y2, a_min=0, a_max=255)
PSNR2 = IT.distortion.PSNR(Y2, X2)
string += rf"], {PSNR2:.2f}"
string += r"~\mathrm{dB}"
string += '$'
plt.title(string) 
#Y = MPG_noise(X, STD_DEV, GAMMA).reshape(X.shape)
#Y2 = utils.generate_MPGN(X2, STD_DEV, GAMMA)
#Y2 = np.clip(a = Y2, a_min=0, a_max=255)
utils.imshow(Y2)
plt.savefig('0MMPG_lake.pdf', bbox_inches='tight')

#### GD_0MMPG_barb

In [ ]:
sigma_kernel = 0.70
sigma = np.array([sigma_kernel, sigma_kernel])
kernel = [None]*2
kernel[0] = utils.get_gaussian_kernel(sigma[0])
kernel[1] = utils.get_gaussian_kernel(sigma[1])
print(np.sum(kernel[0]))
plt.plot(kernel[0])
plt.show()

In [ ]:
denoised1 = denoiser.filter(Y1, kernel)

In [ ]:
PSNR1 = IT.distortion.PSNR(denoised1, X1)

In [ ]:
string  = r"$\mathrm{GD}_{\text{"
string += str(TAU)
string += r"}}(\hat{\mathbf{X}}_1)"
#string += r"}}(\mathrm{Barb} + \mathbf{N}_{\mathcal{N}"
#string += r"(\sigma"
#string += f"={std_dev})"
#string += r'}'
#string += r" + \mathbf{N}_{\mathcal{P}"
#string += r"(\lambda"
#string += f"={gamma}"
#string += r'\cdot\mathrm{Barb})}'
#string += rf"$_{(\sigma^2={std_dev}^2)}$"
#string += f"/{gamma})"
string += f",~{IT.distortion.PSNR(Y1.astype(np.uint8), X1):.2f}"
string += r"~\text{dB}$"
plt.title(f"{string}")
plt.imshow(denoised1, cmap="gray")
plt.savefig('GD_0MMPG_barb.pdf', bbox_inches='tight')

#### GD_0MMPG_lake

In [ ]:
denoised2 = denoiser.filter(Y2, kernel)

In [ ]:
PSNR2 = IT.distortion.PSNR(denoised2, X2)

In [ ]:
string  = r"$\mathrm{GD}_{\text{"
string += str(TAU)
string += r"}}(\hat{\mathbf{X}}_2)"
#string += r"}}(\mathrm{Barb} + \mathbf{N}_{\mathcal{N}"
#string += r"(\sigma"
#string += f"={std_dev})"
#string += r'}'
#string += r" + \mathbf{N}_{\mathcal{P}"
#string += r"(\lambda"
#string += f"={gamma}"
#string += r'\cdot\mathrm{Barb})}'
#string += rf"$_{(\sigma^2={std_dev}^2)}$"
#string += f"/{gamma})"
string += f",~{IT.distortion.PSNR(Y2.astype(np.uint8), X2):.2f}"
string += r"~\text{dB}$"
plt.title(f"{string}")
plt.imshow(denoised2, cmap="gray")
plt.savefig('GD_0MMPG_lake.pdf', bbox_inches='tight')

#### GD_PCC_0MMPG_barb and GD_PCC_0MMPG_lake

In [ ]:
sigmas_kernel = [(0.2 + i/40) for i in range(40)] # Number of points per line
#sigmas_kernel.append(0.625)
#sigmas_kernel.sort()
#sigmas_kernel = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
std_devs = [i for i in range(0, 100, 20)]
PCC_curves1 = []
PCC_curves2 = []
#SFRC_curves = []
poisson_ratio = 0.5
gamma = 0.15
for std_dev in std_devs: # Number of noise levels
    #gamma = std_dev / 50
    PCC_curve1 = []
    PCC_curve2 = []
    #SFRC_curve = []
    Y1 = utils.generate_MPGN(X1, std_dev, gamma, poisson_ratio).reshape(X1.shape)
    Y2 = utils.generate_MPGN(X2, std_dev, gamma, poisson_ratio).reshape(X2.shape)
    #Y = np.clip(a = Y, a_min=0, a_max=255) # Probar a quitar
    #for sigma_kernel in range(5, 20, 1):
    for sigma_kernel in sigmas_kernel: # Filter length
        #sigma_kernel /= 10
        sigma = np.array([sigma_kernel, sigma_kernel])
        kernel = [None]*2
        kernel[0] = utils.get_gaussian_kernel(sigma[0])
        kernel[1] = utils.get_gaussian_kernel(sigma[1])
        #print("Kernel:", kernel)
        denoised1 = denoiser.filter(Y1, kernel)
        denoised2 = denoiser.filter(Y2, kernel)
        #PSNR = IT.distortion.PSNR(denoised, X)
        PCC1 = np.corrcoef(denoised1.flatten(), X1.flatten())[0, 1]
        PCC2 = np.corrcoef(denoised2.flatten(), X2.flatten())[0, 1]
        print("std_dev:", std_dev, "sigma_kernel:", sigma_kernel, "PCC1:", PCC1, "PCC2:", PCC2)
        PCC_curve1.append(PCC1)
        PCC_curve2.append(PCC2)
        #freq, c_avg = fsc.get_SFRC_curve(denoised)
        #first_half = c_avg[:len(c_avg)>>1]
        #SFRC_curve.append(first_half)
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    PCC_curves1.append(PCC_curve1)
    PCC_curves2.append(PCC_curve2)
    #SFRC_curves.append(SFRC_curve)
    
    #sigma_index += 1
    print()

In [ ]:
#plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
#for i in range(len(curves)):
i = 0
for std_dev in std_devs:
    #plt.plot([i/10 for i in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}, \lambda={(10-i)/40}\cdot" + r"\mathrm{Barb}$")
    #if ((10+i*5) == 40) and ((10-i)/40 == 0.15):
    #plt.plot(sigmas_kernel, curves[i], label=rf"$\sigma={10+i*5}, \gamma={(10-i)/40}" + r", \mathrm{argmax}_\tau=" + rf"{sigmas_kernel[np.argmax(curves[i])]:.2f}$", marker='o')
    #else:
    plt.plot(sigmas_kernel,
        PCC_curves1[i],
        label=rf"$\sigma={std_dev}, \gamma={gamma}"
        + r", \tau^*="
        + rf"{sigmas_kernel[np.argmax(PCC_curves1[i])]:.2f}"
        + "$")
    i += 1
string  = r"$"
string += r"\mathrm{PCC}"
string += r"(\mathbf{X}_1, "
string += r"\mathrm{GD}_\tau"
string += r"(\mathbf{X}_1"
string += r"+ \mathbf{N}_{\mathcal{N}"
string += r"(\sigma)}"
string += r"+ \mathbf{N}_"
string += r"{\mathcal{P}(\gamma\mathbf{X}_1)}/\gamma))"
string += r"$"
plt.ylabel(string)
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GD_PCC_0MMPG_barb.pdf", bbox_inches='tight')

In [ ]:
#plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
#plt.title(r"$\mathbb{E}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\mathrm{Barb})$")
#plt.title("Averaging Poisson noisy instances of \"Barb\"")
#for i in range(len(curves)):
i = 0
for std_dev in std_devs:
    #plt.plot([i/10 for i in range(5, 20, 1)], curves[i], label=rf"$\sigma={10+i*5}, \lambda={(10-i)/40}\cdot" + r"\mathrm{Barb}$")
    #if ((10+i*5) == 40) and ((10-i)/40 == 0.15):
    #plt.plot(sigmas_kernel, curves[i], label=rf"$\sigma={10+i*5}, \gamma={(10-i)/40}" + r", \mathrm{argmax}_\tau=" + rf"{sigmas_kernel[np.argmax(curves[i])]:.2f}$", marker='o')
    #else:
    plt.plot(sigmas_kernel,
        PCC_curves2[i],
        label=rf"$\sigma={std_dev}, \gamma={gamma}"
        + r", \tau^*="
        + rf"{sigmas_kernel[np.argmax(PCC_curves2[i])]:.2f}"
        + "$")
    i += 1
string  = r"$"
string += r"\mathrm{PCC}"
string += r"(\mathbf{X}_2, "
string += r"\mathrm{GD}_\tau"
string += r"(\mathbf{X}_2"
string += r"+ \mathbf{N}_{\mathcal{N}"
string += r"(\sigma)}"
string += r"+ \mathbf{N}_"
string += r"{\mathcal{P}(\gamma\mathbf{X}_2)}/\gamma))"
string += r"$"
plt.ylabel(string)
#plt.ylabel(r"$\mathrm{PCC}(\mathbf{X}, \hat{\mathbf{X}})$")
plt.xlabel(r"$\tau$")
plt.legend()
#plt.show()
plt.savefig("GD_PCC_0MMPG_lake.pdf", bbox_inches='tight')